In [1]:
import pandas as pd
import os
from start import data_path
import extract_laws
import re
import spacy
import pandas as pd
import clean_documents

In [2]:
docs = pd.read_csv(os.path.join(data_path, 'text_narrowed.csv'))
docs = clean_documents.remove_whitespace(docs, text_col = 'text')

In [3]:
extract_laws.get_laws(docs[docs['title'] == 'Coupland ISD']['text'].values[0])

[25.0811, 21.003, 21.053, 25.036]

In [4]:
len(docs[docs.text == "Not a plan"])

249

In [5]:
docs['possible_laws'] = docs.text.apply(extract_laws.get_laws)
docs.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,title,level,type,link,text,p_innovation,possible_laws
0,1323,1323,1323,1451,Lake Travis ISD,Second,pdf,https://www.ltisdschools.org//cms/lib/Tx018000...,LTISD Plan of Innovation LAKE TRAVIS INDEPENDE...,0.999945,"[25.0811, 11.251, 21.003, 21.352, 21.102, 21.0..."
1,1913,1913,1913,2108,Zephyr ISD,Second,docx,http://zephyrisd.net/wp-content/uploads/2014/0...,Zephyr ISD District of Innovation Plan Introdu...,0.999955,"[25.036, 21.003, 21.102, 25.0811]"
2,1895,1895,1895,2089,Zavalla ISD,Second,pdf,https://s3.amazonaws.com/scschoolfiles/1772/za...,Zavalla ISD District of Innovation Plan (HB 18...,0.999955,"[25.0811, 25.092, 21.352, 21.401, 21.003, 21.0..."
3,577,577,577,596,Zapata County ISD,Second,pdf,https://1.cdn.edl.io/VZ6fi1M6pn8Qk8tS3GetGewCD...,Not a plan,0.249567,[]
4,1784,1784,1784,1967,Yorktown ISD,Second,pdf,http://www.yisd.org/userfiles/57/my%20files/fi...,Yorktown Independent School District Final Dis...,0.999955,"[25.082, 25.081, 21.053, 21.003, 21.102, 25.08..."


In [6]:
docs.to_csv(os.path.join(data_path, 'doi_exemptions_scraped.csv'))
pd.options.display.max_colwidth = 10000
docs[docs.title == "Blum ISD"].text

757    BLUM Independent School District District of Innovation Plan June 2017-May 2022 1 A District of Innovation is a concept passed by the 84th Legislative Session in House Bill 1842 that allows school districts greater local control and flexibility regarding certain state-level regulations in an effort to utilize the designation to better serve students. The proposed plan, once adopted, will remain in effect for the next five years (June 2017-May 2022). The plan may be amended at any time by the District of Innovation Committee with the approval of the school board. District of Innovation Committee Jeff Sanders Administration Kim Moore Middle School Traci Bellomy Administration Cooper Thornhill High School Mark Brandenberger Administration Leah Davis High School Jody Fergason Administration LaDonna Jean High School Tiffany Quinn Elementary Samantha Maxwell Parent Jennifer Williams Elementary LaDonna Poteet Parent Chantelle Hurt Middle School Elsa Scott Community Timeline Task Feb. 2

## Validate laws

In [7]:
test = docs[docs.p_innovation > .9].sample(30, random_state = 117)
test['missing_laws'] = ''
test['incorrect_laws'] = ''
test[['title', 'link', 'possible_laws', 'incorrect_laws']].to_csv((os.path.join(data_path, 'sample_laws.csv')))

In [8]:
results = pd.read_csv(os.path.join(data_path, 'sample_laws_filledin.csv'))

In [9]:
results.head()

,Unnamed: 0,title,link,possible_laws,incorrect_laws,gold_standard
0,552,Evant ISD,https://s3.amazonaws.com/scschoolfiles/1553/evant_district_of_innovation_plan-6-5-18b.pdf,"[21.401, 25.081, 25.081, 21.003, 25.0811, 21.003]",0.0,"[21.401, 25.081, 25.081, 21.003, 25.0811, 21.003]"
1,687,Buna ISD,https://s3.amazonaws.com/scschoolfiles/130/buna_isd_doi_plan__04102017.pdf,"[25.0811, 21.003, 21.057, 25.0812, 21.053, 25.113, 21.057, 21.458, 21.055, 25.113, 21.458, 21.055, 21.451, 21.053, 21.003, 25.0811, 25.112, 21.451, 25.112, 25.113, 25.0812]",1.0,"[25.0811, 25.0812, 25.112, 21.003, 21.053, 21.055, 21.057, 21.102, 21.451, 21.458]"
2,759,Anson ISD,"http://www.ansontigers.com/users/2016-17/District%20of%20Innovation,%202017.pdf","[25.092, 21.003, 21.102, 25.081, 21.458, 21.003, 11.253, 28.0214, 21.404, 21.458, 21.404, 25.081, 25.0811, 21.057, 21.057, 28.0216, 25.092, 21.102, 21.003, 25.092, 11.253]",2.0,NaN
3,31,White Deer ISD,http://s3.amazonaws.com/scschoolfiles/1380/white_deer_isd_plan_of_innovation.pdf,"[21.057, 25.0811, 21.003, 28.004, 25.0811, 21.003, 25.036, 21.053, 25.001, 37.008, 25.036, 28.004, 21.401, 28.004, 25.082, 25.0811, 37.008, 25.036, 25.082]",0.0,"[25.0811, 21.003, 21.053, 21.057, 21.401, 25.082, 37.008, 25.036]\n"
4,692,Bryan ISD,https://1.cdn.edl.io/3fzDFgsiJY1q2kNSwlpWOc5xqOFR6E9ke5pjwMiuzMyJnpsf.pdf,"[25.0811, 25.113, 25.092, 21.057, 28.0216, 25.112, 25.113, 25.112, 25.092, 25.092, 25.0811, 25.111, 25.111, 28.214]",2.0,"[25.0811, 21.003, 25.092, 25.111, 25.112, 25.113]"
